In [22]:
'''
神經網路 (neural network)
    感知機器 (perceotion)
'''
from collections import Counter
from functools import partial
from linear_algebra import dot
import math, random
import matplotlib
import matplotlib.pyplot as plt

# 感知機器
# 可接受 n 個二元輸入的單一神經元所構成
# 加權總和大於或等於零
def step_function(x):
    return 1 if x >= 0 else 0

def perception_output(weights, bias, x):
    '''
    如果感知機器要做一進一步反應，就送回 1，不然就送回 0
    '''
    calculation = dot(weights, x) + bias
    return step_function(calculation)

# 利用超平面 (hyperplane)把點 x 切分到兩個半空間之中
dot(weights, x) + bias == 0

# 建立一個 AND閘
# all of 1, equal to 1
weights = [2, 2]
bias = -3

# 建立一個 OR閘
weights = [2, 2]
bias = -1

# 建立一個 NOT閘
# 1-> 0, 0-> 1
weights = [-2]
bias = 1

ModuleNotFoundError: No module named 'linear_algebra'

In [24]:
# 單純做邏輯閘
and_gate = min
or_gate = max
xor_gate = lambda x, y: 0 if x == y else 1

In [27]:
# 正向饋送 (feed-forward) 神經網路
# S 型函數取代階梯函數
def sigmoid(t):        # 是一種不錯的平滑函數
    return 1 / (1 + math.exp(-t))

def neuron_output(weights, inputs):
    return sigmoid(dot(weights, inputs))

def feed_forward(neural_network, input_vector):
    '''
    以一個神經網路 neural_network 做為輸入
    (以權重的列表的列表的列表來表示)
    並根據正向傳播的輸入 input_vector 送出輸出的結果
    '''
    
    outputs = []
    
    # 一次處理一層
    for layer in neural_network:
        input_with_bias = input_vector + [1]
        output = [neuron_output(neuron, input_with_bias)
                 for neuron in layer]
        outputs.append(output)
        
        # 然後下一層的輸入，就是這一層的輸出
        input_vector = output
    return outputs

In [28]:
xor_network =[          # 隱藏層
    [[20, 20, -30],     # 「AND」神經元
     [20, 20, -10]],    # 「OR」神經元
    # 輸入層
    [[-60, 60, -30]]    # 「第二輸入 AND NOT 第一輸入」神經元
]

for x in [0, 1]:
    for y in [0, 1]:
        # feed_forward 會製作出每個神經元的輸出
        # feed_forward[-1] 則是輸出層神經元的輸出
        print(x, y, feed_forward(xor_network, [x, y])[-1])
        
# 0 0 [9.38314668300676e-14]
# 0 1 [0.9999999999999059]
# 1 0 [0.9999999999999059]
# 1 1 [9.383146683006828e-14]

NameError: name 'dot' is not defined

In [35]:
# 反向傳播 (backpropagate)
def backpropagate(network, input_vector, targets):
    hidden_outputs, outputs = feed_forward(network, input_vector)
    
    # output * (1 - output) 是來自 S 型函數的導函數
    output_deltas = [output * (1 - output) * (output - target)
                    for output, target in zip(outpust, targets)]
    
    # 調整輸出層的權重，一次一個神經元
    for i, output_neron in enumerate(network[-1]):
        # 聚焦於第 i 個輸出神經元
        for j, hidden_output in enumerate(hidden_outputs + [1]):
            # 調整第 j 個權重值，根據的是
            # 此神經元的 delta，以及它的第 j 個輸入
            output_neuron[j] -= output_deltas[i] * hidden_output
            
    # 把誤差反向傳播至隱藏層
    hidden_deltas = [hidden_output * (1 - hidden_output) * 
                    dot(output_deltas, [n[i] for n in output_layer])
                    for i, hidden_output in enumerate(hidden_outputs)]
    
    # 調整隱藏層的權重值，一次一個神經元
    for i, hidden_neuron in enumerate(network[0]):
        for j, input in enumerate(input_vector + [1]):
            hidden_neuron[j] -= hidden_deltas[i] * input

In [36]:
if __name__ == "__main__":

    raw_digits = [
          """11111
             1...1
             1...1
             1...1
             11111""",

          """..1..
             ..1..
             ..1..
             ..1..
             ..1..""",

          """11111
             ....1
             11111
             1....
             11111""",

          """11111
             ....1
             11111
             ....1
             11111""",

          """1...1
             1...1
             11111
             ....1
             ....1""",

          """11111
             1....
             11111
             ....1
             11111""",

          """11111
             1....
             11111
             1...1
             11111""",

          """11111
             ....1
             ....1
             ....1
             ....1""",

          """11111
             1...1
             11111
             1...1
             11111""",

          """11111
             1...1
             11111
             ....1
             11111"""]

In [37]:
import random

zero_digit = [1, 1, 1, 1, 1,
             1, 0, 0, 0, 1,
             1, 0, 0, 0, 1,
             1, 0, 0, 0, 1,
             1, 1, 1, 1, 1]

# 如果輸出 10個數字，認為是數字 4
[0, 0, 0, 0, 1, 0, 0, 0, 0, 0]

def make_digit(raw_digit):
    return [1 if c == '1' else 0
            for row in raw_digit.split("\n")
            for c in row.strip()]

inputs = list(map(make_digit, raw_digits))

targets = [[1 if i == j else 0 for i in range(10)]
          for j in range(10)]        # target[4] 表示數字 4

random.seed(0)      # 可取得重複結果
input_size = 25     # 每個輸入都是一個長度為 25的向量
num_hidden = 5      # 有 5 個神經元會放在隱藏層中
output_size = 10    # 每個輸入都需要 10個輸出

# 每個隱藏神經元對應每個輸入都有個相應的權重，再加上一個偏差權重
hidden_layer = [[random.random() for __ in range(input_size + 1)]
                    for __ in range(num_hidden)]

# 每個輸入神經元對應每個隱藏神經元都有個權重，再加上一個偏差權重
output_layer = [[random.random() for __ in range(num_hidden + 1)]
                for __ in range(output_size)]

# 網路一開始是以一組隨機權重作為起始值
network = [hidden_layer, output_layer]

# 10,000 次的迭代，似乎已足以得到收斂的結果
for __ in range(10000):
    for input_vector, target_vector in zip(inputs, targets):
        backpropagate(network, input_vector, target_vector)

NameError: name 'dot' is not defined

In [38]:
 def predict(input):
        return feed_forward(network, input)[-1]
    
print(inputs[7])
# [0.026, 0.0, 0.0, 0.018, 0.001, 0.0, 0.0, 0.967, 0.0, 0.0]

[1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1]


In [43]:
# for i, input in enumerate(inputs):
#     outputs = predict(input)
#     print(i, [round(p,2) for p in outputs])

print(""".@@@.
...@@
..@@.
...@@
.@@@.""")

print([round(x, 2) for x in
      predict(  [0,1,1,1,0,    # .@@@.
                 0,0,0,1,1,    # ...@@
                 0,0,1,1,0,    # ..@@.
                 0,0,0,1,1,    # ...@@
                 0,1,1,1,0])]) # .@@@.
print()

# [0.0, 0.0, 0.0, 0.92, 0.0, 0.0, 0.0, 0.01, 0.0, 0.12]

.@@@.
...@@
..@@.
...@@
.@@@.


NameError: name 'dot' is not defined

In [46]:
print(""".@@@.
@..@@
.@@@.
@..@@
.@@@.""")

print([round(x, 2) for x in
      predict(  [0,1,1,1,0,    # .@@@.
                 1,0,0,1,1,    # @..@@
                 0,1,1,1,0,    # .@@@.
                 1,0,0,1,1,    # @..@@
                 0,1,1,1,0])]) # .@@@.
print()

# [0.0, 0.0, 0.0, 0.0, 0.0, 0.55, 0.0, 0.0, 0.93, 1.0]

.@@@.
@..@@
.@@@.
@..@@
.@@@.


NameError: name 'dot' is not defined

In [53]:
# pyplot.imshow 製作這些圖片

from matplotlib import pyplot as plt

weights = network[0][0]                   # 隱藏層中的第一神經元
abs_weights = list(map(abs, weights))           # 只根據絕對值來決定暗度

grid = [abs_weights[row:(row+5)]          # 把權重值轉換為 5*5 的方格圖
       for row in range(0, 25, 5)]        # [weights[0:5], ..., weights[20:25]]

ax = plt.gca()                            # 為了畫出交叉陰陽線，需要用到座標軸

ax.imshow(grid,                           # 用法和 plt.imshow
         cmap=matplotlib.cm.binary,       # 用黑白灰階來表示
         interpolation='none')            # 一塊一塊畫，不做插補

def patch(x, y, hatch, color):
    '''
    送回一個 matplotlib 'patch' 物件，其中帶有指定的
    位置、交叉陰影線圖樣，以及顏色
    '''
    return matplotlib.patches.Rectangle((x - 0.5, y - 0.50), 1, 1,
                                       hatch=hatch, fill=False, color=color)

# 負值權重以交叉陰影線表示
for i in range(5):                        # 列 (row)
    for j in range(5):                    # 行 (column)
        if weights[5*i + j] < 0:          # 第 i 列第 j 行 = weights[5*i + j]
            # 添加黑白交叉陰影線，不管顏色明暗都看的見
            ax.add_patch(patch(j, i, '/', "white"))
            ax.add_patch(patch(j, i, '\\', "black"))
plt.show()

NameError: name 'matplotlib' is not defined

In [56]:
# 第一個隱藏神經元的輸出
left_column_only = [1, 0, 0, 0, 0] * 5
print(feed_forward(network, left_column_only)[0][0])   # 1.0

center_middle_row = [0, 0, 0, 0, 0] * 2 + [0, 1, 1, 1, 0] + [0, 0, 0, 0, 0] *2
print(feed_forward(network, center_middle_row)[0][0])  # 0.95

right_column_only = [0, 0, 0, 0, 1] * 5
print(feed_forward(network, right_column_only)[0][0])  # 0.0

NameError: name 'dot' is not defined

In [59]:
# 數字 3 放入神經網路
my_three = [0, 1, 1, 1, 0,   # .@@@.
           0, 0, 0, 1, 1,    # ...@@
           0, 0, 1, 1, 0,    # ..@@.
           0, 0, 0, 1, 1,    # ...@@
           0, 1, 1, 1, 0]    # .@@@.

hidden, output = feed_forward(network, my_three)

0.121080   # 取自 network[0][0]，可能受到 (1, 4)負值的影響
0.999979   # 取自 network[0][1]，受到 (0, 2) 和 (2, 2)正值的影響很大
0.999999   # 取自 network[0][2]，除了 (3, 4)以外都是正值
0.999992   # 取自 network[0][3]，再次受到 (0, 2) 和 (2, 2)正值的很大影響
0.000000   # 取自 network[0][4]，除了中間的橫列之外，其他非零即負

# 權重值 network[-1][3]
# -11.61     # hidden[0] 的權重值
# - 2.17     # hidden[1] 的權重值
#   9.31     # hidden[2] 的權重值
# - 1.38     # hidden[3] 的權重值
# -11.47     # hidden[4] 的權重值
# - 1.92     # 偏差輸入的權重值

# 神經元的計算
sigmoid(0.121 * -11.61 + 1 * -2.17 + 1 * 9.31 -1.38 * 1 - 0 * 11.47 -1.92)

NameError: name 'dot' is not defined